# Experimentos auxiliares para el TFG

Este notebook recoge una serie de bloques de código utilizados como apoyo durante el desarrollo del Trabajo de Fin de Grado. Incluye tareas complementarias como la traducción automática de reseñas al inglés, la generación de embeddings textuales con BERT, el cálculo de similitudes entre imágenes y textos con CLIP y el procesamiento por lotes de datos para evitar cuelgues del sistema. Aunque no forman parte del pipeline principal del sistema de recomendación, estos experimentos y utilidades permitieron tomar decisiones clave sobre qué representaciones utilizar y cómo preprocesar los datos de forma eficiente.

# Traducción Automática de Reseñas con MarianMT

Este bloque traduce automáticamente los textos de reseñas de restaurantes del español al inglés utilizando
el modelo MarianMT de Helsinki-NLP. El objetivo principal es adaptar los datos a modelos de recomendación
multimodal que utilizan embeddings generados con CLIP.

CLIP fue entrenado mayoritariamente con datos en inglés, por lo que se sospecha que las arquitecturas
que usan embeddings CLIP de texto funcionarán mejor si las reseñas están traducidas al inglés. Estas
arquitecturas incluyen:

- CLIPTextRegressor: utiliza solo embeddings CLIP de texto.
- CLIPMixedRegressor y FullModel: combinan embeddings CLIP de texto e imagen con identificadores de usuario e ítem.
- MixedModel: mezcla codificación tradicional (IDs) con embeddings de texto CLIP.

Traducir los textos antes de generar los embeddings puede mejorar el rendimiento de estas arquitecturas,
al alinear mejor el lenguaje de entrada con el vocabulario aprendido por CLIP durante su entrenamiento.

In [ ]:

# ------------------------------
# Importación de librerías
# ------------------------------

import pandas as pd                 
from tqdm.auto import tqdm           
import os                            
import torch                        

# Modelos y pipeline de traducción
from transformers import MarianMTModel, MarianTokenizer, pipeline 

# ------------------------------
# Comprobación del entorno
# ------------------------------

# Indica si hay GPU disponible para acelerar el proceso
print("GPU disponible:", torch.cuda.is_available())

# ------------------------------
# 1) Carga del modelo y tokenizer
# ------------------------------

# Nombre del modelo específico de español a inglés de Helsinki-NLP
model_name = "Helsinki-NLP/opus-mt-es-en"

# Se carga el tokenizer con opción rápida (más eficiente)
tokenizer = MarianTokenizer.from_pretrained(model_name, use_fast=True)

# Se carga el modelo y se transfiere a GPU (cuda)
model = MarianMTModel.from_pretrained(model_name).to("cuda")

# ------------------------------
# 2) Creación del pipeline de traducción
# ------------------------------

# Se construye el pipeline indicando explícitamente:
# - Modelo y tokenizer
# - Uso de GPU con device=0 (primera GPU disponible)
# - batch_size grande para acelerar, si hay suficiente VRAM
# - max_length y num_beams ajustados para rapidez
translator = pipeline(
    "translation",
    model=model,
    tokenizer=tokenizer,
    device=0,               # GPU 0
    framework="pt",         # PyTorch
    batch_size=64,
    max_length=512,
    num_beams=1,            # búsqueda sin beams para mayor velocidad
    do_sample=False         # desactiva muestreo aleatorio
)

# ------------------------------
# 3) Función de traducción por batches
# ------------------------------

def translate_dataset_batched(name, batch_size=64, max_chars=512):
    """
    Traduce un dataset .pkl con una columna 'text' del español al inglés.
    Guarda el resultado en otro archivo con sufijo '_en.pkl'.

    Parámetros:
    - name: nombre base del archivo (sin .pkl)
    - batch_size: número de ejemplos a traducir por lote
    - max_chars: longitud máxima de texto por entrada (truncado)
    """

    inp = f"{name}.pkl"
    out = f"{name}_en.pkl"

    print(f"\n>> Cargando {inp}…")
    df = pd.read_pickle(inp)

    # Se obtienen los textos, se rellenan nulos y se convierten a string
    texts = df["text"].fillna("").astype(str).tolist()

    # Truncado previo para asegurar compatibilidad con el modelo
    texts = [
        (t if len(t) <= max_chars else t[:max_chars] + "...")
        for t in texts
    ]

    # Traducción por lotes
    translated = []
    for i in tqdm(range(0, len(texts), batch_size), desc=f"Traduciendo {name}"):
        batch = texts[i : i + batch_size]
        outputs = translator(batch)
        translated.extend([o["translation_text"] for o in outputs])

    # Se añade la columna traducida al DataFrame
    df["text_en"] = translated

    print(f">> Guardando en {out}…")
    df.to_pickle(out)
    print(f"✅ {name} traducido y guardado.")

# ------------------------------
# 4) Traducción de datasets disponibles
# ------------------------------

# Se aplica la función a los conjuntos de entrenamiento, validación y prueba
for split in ["train_strict", "val_strict", "test_strict"]:
    if os.path.isfile(f"{split}.pkl"):
        translate_dataset_batched(split)
    else:
        print(f"[SKIP] No existe {split}.pkl")

# Generación de Embeddings Textuales con BERT

Este bloque genera representaciones vectoriales (embeddings) a partir de reseñas escritas en texto,
utilizando el modelo preentrenado BERT (bert-base-uncased). A diferencia de CLIP, que genera
embeddings multimodales (texto + imagen), BERT está optimizado exclusivamente para lenguaje natural,
por lo que puede capturar mejor las relaciones semánticas presentes en las reseñas.


Durante los experimentos se observó que las imágenes asociadas a los restaurantes no aportaban información relevante para la tarea de recomendación. En cambio, las reseñas textuales demostraron tener un alto poder predictivo. Por ello, se decidió probar un modelo alternativo especializado en lenguaje natural: BERT, con la hipótesis de que generaría embeddings más representativos que CLIP.

Los resultados confirmaron esta sospecha: los embeddings generados con BERT ofrecieron un mejor rendimiento que los obtenidos con CLIP. Esto sugiere que, en este dominio concreto, el contenido lingüístico de las reseñas es el principal factor que determina la satisfacción del usuario.

El objetivo de este script es, por tanto, reemplazar los embeddings generados con CLIP por embeddings obtenidos con BERT, que se almacenan en una nueva columna llamada "text_emb" dentro del DataFrame.

In [ ]:
# ------------------------------
# 0) Instalación de librerías
# ------------------------------
# Se instalan Transformers y TensorFlow (aunque este script no necesita TF)
# En entornos como Kaggle o Colab, TensorFlow puede generar conflictos si no se usa
!pip install -q tensorflow tensorflow-estimator
!pip install -q transformers

# ------------------------------
# 1) Importación de librerías
# ------------------------------

import pandas as pd                            # Manipulación de DataFrames
import numpy as np                             # Operaciones numéricas
import torch                                   # Uso de GPU y modelo BERT
from transformers import AutoTokenizer, AutoModel  # Carga del modelo BERT
from tqdm.auto import tqdm                     # Barra de progreso

# ------------------------------
# 2) Configuración del entorno
# ------------------------------

device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU si está disponible
MODEL_NAME = "bert-base-uncased"   # Modelo preentrenado de BERT (casing ignorado)
BATCH_SIZE = 32                    # Procesamiento por lotes
MAX_LEN    = 128                   # Máximo número de tokens por texto

# ------------------------------
# 3) Carga de modelo y tokenizador
# ------------------------------

# Tokenizador que divide el texto en subpalabras y añade tokens especiales
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Modelo BERT que genera los embeddings
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()  # Se desactiva el modo entrenamiento (no dropout, no actualiza pesos)

# ------------------------------
# 4) Mean Pooling
# ------------------------------

def mean_pooling(hidden_states, attention_mask):
    """
    Calcula el embedding medio de las palabras reales (excluyendo padding).
    
    hidden_states: salida de BERT (batch_size, seq_len, hidden_dim)
    attention_mask: máscara con 1s en posiciones válidas y 0s en padding
    """
    mask = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
    return (hidden_states * mask).sum(1) / mask.sum(1)  # Promedio solo donde mask=1

# ------------------------------
# 5) Función para obtener embeddings BERT
# ------------------------------

def get_bert_embeddings(texts, desc=""):
    """
    Procesa una lista de textos, tokeniza y genera embeddings con BERT.
    
    Parámetros:
    - texts: lista de cadenas de texto
    - desc: descripción para mostrar en la barra de progreso

    Devuelve:
    - matriz NumPy (N, hidden_dim) con los embeddings
    """
    all_embs = []

    for i in tqdm(range(0, len(texts), BATCH_SIZE), desc=f"📝 Embedding {desc}"):
        batch = texts[i : i + BATCH_SIZE]

        # Tokenización por lotes con padding y truncado
        enc = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=MAX_LEN,
            return_tensors="pt"
        )

        input_ids      = enc["input_ids"].to(device)
        attention_mask = enc["attention_mask"].to(device)

        # Cálculo de embeddings sin actualizar el modelo
        with torch.no_grad():
            out = model(input_ids=input_ids, attention_mask=attention_mask)

        # Aplicar mean pooling
        emb = mean_pooling(out.last_hidden_state, attention_mask)

        # Convertir a NumPy y guardar
        all_embs.append(emb.cpu().numpy())

    return np.vstack(all_embs)  # (N, dim)

# ------------------------------
# 6) Procesamiento de archivos
# ------------------------------

# Diccionario con rutas a los archivos .pkl de cada split
FILES = {
    "train": "/kaggle/input/ny8010/train_v80.pkl",
    "val":   "/kaggle/input/ny8010/val_v10.pkl",
    "test":  "/kaggle/input/ny8010/test_v10.pkl"
}

# Proceso por cada split
for split, path in FILES.items():
    print(f"\n🔄 Procesando {split} ({path})…")

    # Carga del DataFrame
    df = pd.read_pickle(path)

    # Normalización de textos: asegurar strings y eliminar NaNs
    df["text"] = df["text"].fillna("").astype(str)

    # Obtener embeddings
    embs = get_bert_embeddings(df["text"].tolist(), split)

    # Añadir al DataFrame
    df["text_emb_bert"] = list(embs)

    # Guardar archivo enriquecido
    out = f"/kaggle/working/{split}_bert_emb.pkl"
    df.to_pickle(out)
    print(f"✅ Guardado embeddings en {out}")


### Arquitectura con la que se verefica que BERT da mejor resultado que CLIP

Se probó la arquitectura 4-MIX-TXT, que combina los identificadores de usuario e ítem con los embeddings textuales de las reseñas. Inicialmente, esta arquitectura se entrenó utilizando embeddings generados con CLIP. Sin embargo, los resultados no fueron del todo satisfactorios: el mejor RMSE en validación fue de 6.68 para el conjunto de Nueva York.

Con el objetivo de mejorar la calidad de los embeddings textuales, se sustituyó CLIP por BERT (bert-base-uncased), un modelo especializado en lenguaje natural. Esta modificación tuvo un impacto positivo claro: el mejor RMSE en validación descendió a 6.03 en Nueva York, una mejora significativa en la precisión del sistema de recomendación.

Este experimento demuestra que, en esta tarea concreta, las imágenes no aportan valor predictivo, y que el texto —representado mediante embeddings BERT— resulta ser el componente clave. Por tanto, reemplazar CLIP por BERT mejora la capacidad del modelo para anticipar la satisfacción del usuario.

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

# ========================
# CONFIGURACIÓN GENERAL
# ========================
start_total = time.time()

DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"
use_amp    = DEVICE == "cuda"
EMB_DIM    = 50                 # dimensión de user/item embeddings
BATCH_SIZE = 128
LR_LIST    = [1e-5, 1e-4, 5e-4, 1e-3]
MAX_EPOCHS = 1000
PATIENCE   = 10
WD         = 1e-6               # weight_decay
RESULTS_XLSX = "Arquitectura_4_MIX_TXTEN_NY_BERT.xlsx"

# ========================
# DATASETS
# ========================
class MixedDataset(Dataset):
    def __init__(self, df):
        # usuario, restaurante
        self.u = torch.tensor(df['user_id_new'].values, dtype=torch.long)
        self.i = torch.tensor(df['restaurant_id_new'].values, dtype=torch.long)
        # embeddings BERT de texto (dim 768)
        self.x = torch.tensor(np.vstack(df['text_emb'].values), dtype=torch.float32)
        # rating
        self.r = torch.tensor(df['rating'].values, dtype=torch.float32)
    def __len__(self):
        return len(self.r)
    def __getitem__(self, idx):
        return self.u[idx], self.i[idx], self.x[idx], self.r[idx]

# cargar DataFrames
train = pd.read_pickle("/kaggle/input/ny-bert/train_bert_emb.pkl")
val   = pd.read_pickle("/kaggle/input/ny-bert/val_bert_emb.pkl")
test  = pd.read_pickle("/kaggle/input/ny-bert/test_bert_emb.pkl")

# crear DataLoaders
train_loader = DataLoader(MixedDataset(train), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(MixedDataset(val),   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(MixedDataset(test),  batch_size=BATCH_SIZE, shuffle=False)

n_users = train['user_id_new'].nunique()
n_items = train['restaurant_id_new'].nunique()
txt_dim = train['text_emb'].iloc[0].shape[0]  # debería ser 768

# ========================
# MODELO MIX-TXTEN
# ========================
class MixedModel(nn.Module):
    def __init__(self, n_users, n_items, txt_dim, emb_dim=EMB_DIM):
        super().__init__()
        # embeddings clásicos user/item
        self.uemb = nn.Embedding(n_users, emb_dim)
        self.iemb = nn.Embedding(n_items, emb_dim)
        nn.init.normal_(self.uemb.weight, mean=1.0, std=0.01)
        nn.init.normal_(self.iemb.weight, mean=1.0, std=0.01)
        # MLP final: concat [uemb | iemb | text_emb]
        self.mlp = nn.Sequential(
            nn.Linear(2*emb_dim + txt_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, u, i, txt):
        ue = self.uemb(u)
        ie = self.iemb(i)
        x = torch.cat([ue, ie, txt], dim=1)
        return self.mlp(x).squeeze(1)

# ========================
# MÉTRICA RMSE
# ========================
def compute_rmse(model, loader, criterion):
    model.eval()
    total, n = 0.0, 0
    with torch.no_grad():
        for u,i,txt,r in loader:
            u,i,txt,r = u.to(DEVICE), i.to(DEVICE), txt.to(DEVICE), r.to(DEVICE)
            pred = model(u,i,txt)
            loss = criterion(pred, r)
            total += loss.item() * r.size(0)
            n += r.size(0)
    return np.sqrt(total / n)

# ========================
# ENTRENAMIENTO
# ========================
results = []
criterion = nn.MSELoss()

for lr in LR_LIST:
    print(f"\nEntrenando MIX-TXTEN @ lr={lr}")
    t0 = time.time()

    model = MixedModel(n_users, n_items, txt_dim).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=WD)
    scaler = torch.cuda.amp.GradScaler() if use_amp else None

    best_val, best_epoch, best_state = float('inf'), 0, None
    train_hist, val_hist = [], []
    early_cnt = 0

    for epoch in tqdm(range(1, MAX_EPOCHS+1), desc=f"MIX lr={lr}"):
        model.train()
        running, count = 0.0, 0
        for u,i,txt,r in train_loader:
            u,i,txt,r = u.to(DEVICE), i.to(DEVICE), txt.to(DEVICE), r.to(DEVICE)
            optimizer.zero_grad()
            if use_amp:
                with torch.cuda.amp.autocast():
                    pred = model(u,i,txt)
                    loss = criterion(pred, r)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                pred = model(u,i,txt)
                loss = criterion(pred, r)
                loss.backward()
                optimizer.step()
            running += loss.item() * r.size(0)
            count   += r.size(0)

        train_rmse = np.sqrt(running/count)
        val_rmse   = compute_rmse(model, val_loader, criterion)
        train_hist.append(train_rmse)
        val_hist.append(val_rmse)

        if val_rmse < best_val:
            best_val, best_epoch, best_state = val_rmse, epoch, model.state_dict()
            early_cnt = 0
        else:
            early_cnt += 1
            if early_cnt >= PATIENCE:
                print(f"Early stopping at epoch {epoch} (best {best_epoch})")
                break

    # restaurar mejor
    model.load_state_dict(best_state)
    test_rmse = compute_rmse(model, test_loader, criterion)
    dt = (time.time() - t0)/60

    results.append({
        "Arquitectura":  "4-MIX-TXTEN-NY",
        "lr":            lr,
        "train_rmse":    train_hist[best_epoch-1],
        "val_rmse":      best_val,
        "test_rmse":     test_rmse,
        "epochs":        best_epoch,
        "train_time_m":  round(dt,2)
    })

    # curva
    plt.figure(figsize=(4,3))
    plt.plot(train_hist, label="Train RMSE")
    plt.plot(val_hist,   label="Val RMSE")
    plt.axvline(best_epoch, ls="--", c="gray", label="Best ep.")
    plt.title(f"MIX-TXTEN @ lr={lr}")
    plt.xlabel("Epoch"); plt.ylabel("RMSE")
    plt.legend(loc="upper right", fontsize="small")
    plt.tight_layout()
    plt.savefig(f"curve_mix_txten_lr{lr}.pdf")
    plt.close()

# guardar resultados
df_res = pd.DataFrame(results)
df_res.to_excel(RESULTS_XLSX, index=False)
print(f"\nResultados en {RESULTS_XLSX}")
print(f"Tiempo total: {(time.time()-start_total)/60:.1f} min")


# Similitud imagen-texto en español e inglés para Gijón mediante CLIP

En este análisis se ha comparado la similitud entre los embeddings de imagen y los de texto generados por CLIP, tanto en español como en inglés. A pesar de que CLIP fue entrenado principalmente en inglés, los resultados muestran que, en esta tarea, los textos en español presentan una mayor similitud con las imágenes. Esto sugiere que, cuando se trabaja con entradas multimodales (imagen + texto), puede ser preferible mantener el idioma original de las reseñas, ya que la traducción introduce variaciones que reducen la alineación semántica con el contenido visual.

Sin embargo, cuando se utiliza únicamente el texto como entrada, los experimentos muestran que los embeddings generados a partir de las reseñas traducidas al inglés permiten una mejor predicción del rating. Esto puede deberse a que CLIP y otros modelos preentrenados como BERT han sido optimizados en gran medida sobre corpora en inglés, por lo que su capacidad de comprensión y representación del lenguaje es más fina en ese idioma.

En resumen:

- Para tareas multimodales (imagen + texto): los textos en español se alinean mejor con las imágenes.

- Para tareas solo textuales: las reseñas traducidas al inglés producen embeddings más efectivos para la predicción.

Esto refuerza la importancia de adaptar la representación del texto en función del tipo de entrada del modelo y del objetivo concreto de la tarea.

In [ ]:
# Importación de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

# 1) Filtrar filas con embeddings válidos
mask_valid = (
    train_df['image_emb'].apply(lambda x: isinstance(x, (list, np.ndarray))) &
    train_df['text_emb_es'].apply(lambda x: isinstance(x, (list, np.ndarray))) &
    train_df['text_emb_en'].apply(lambda x: isinstance(x, (list, np.ndarray)))
)
df2 = train_df[mask_valid].reset_index(drop=True)

# 2) Calcular similitudes
def compute_similarities(row):
    img    = np.array(row['image_emb'], dtype=np.float32)
    txt_es = np.array(row['text_emb_es'], dtype=np.float32)
    txt_en = np.array(row['text_emb_en'], dtype=np.float32)
    sim_es = cosine_similarity(img[None], txt_es[None])[0, 0]
    sim_en = cosine_similarity(img[None], txt_en[None])[0, 0]
    return pd.Series({'Español': sim_es, 'Inglés': sim_en})

# 3) Aplicar función
sims = df2.apply(compute_similarities, axis=1)
df_sims = pd.concat([df2[['reviewId']], sims], axis=1)

# 4) Resumen estadístico
df_sims['en_better'] = df_sims['Inglés'] > df_sims['Español']
summary = {
    'Mean sim_es': df_sims['Español'].mean(),
    'Mean sim_en': df_sims['Inglés'].mean(),
    'Proportion sim_en > sim_es': df_sims['en_better'].mean()
}
print(pd.DataFrame([summary]))

# 5) Boxplot
plt.figure(figsize=(6, 4))
df_melted = df_sims.melt(value_vars=['Español', 'Inglés'], var_name='Idioma', value_name='Similitud')
sns.boxplot(x='Idioma', y='Similitud', data=df_melted, palette=["#4B6C8F", "#9FB4C7"])
# Título: Distribución de similitud imagen-texto por idioma
plt.tight_layout()
plt.savefig("clip_boxplot_sim_idiomas.pdf", format="pdf")
plt.show()

# Ejemplo de cálculo de la similitud imagen-texto con CLIP

En este bloque se analiza la alineación semántica entre imágenes y sus descripciones textuales mediante el modelo CLIP.

La matriz muestra las similitudes del coseno entre los embeddings generados por CLIP para cinco imágenes y cinco textos correspondientes. Se observa que los valores de la diagonal principal son los más altos en cada fila, lo que indica que cada imagen se alinea mejor con su descripción original. Esto confirma que CLIP es capaz de emparejar correctamente texto e imagen en estos ejemplos.

Además, fuera de la diagonal, destacan algunas similitudes elevadas como la de "Coliseum" y "Stadium" (0.2712 y 0.2385), lo cual es coherente ya que ambos conceptos están semánticamente relacionados como estructuras arquitectónicas para eventos. Este patrón sugiere que el modelo también capta relaciones interconceptuales más allá del emparejamiento exacto, mostrando cierta capacidad de generalización.

En conjunto, la matriz valida el comportamiento esperado del modelo: puntuaciones más altas cuando la imagen y el texto corresponden, y puntuaciones moderadas o bajas en el resto, salvo en casos con cierta cercanía semántica.

In [ ]:
# Importación de librerías
import torch
import clip
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# Cargar modelo CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Frases y sus rutas de imágenes
texts = [
    "a photo of black seafood rice",
    "a photo of a dog",
    "a photo of some chips",
    "a photo of the coliseum",
    "a photo of a stadium"
]

image_paths = [
    "/kaggle/input/clipexamples/black_seafood_rice.jpg",
    "/kaggle/input/clipexamples/dog.jpg",
    "/kaggle/input/clipexamples/chips.jpg",
    "/kaggle/input/clipexamples/coliseum.jpg",
    "/kaggle/input/clipexamples/stadium.jpg"
]

# Codificar textos
text_tokens = clip.tokenize(texts).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_tokens)

# Codificar imágenes
image_features = []
for path in image_paths:
    image = preprocess(Image.open(path)).unsqueeze(0).to(device)
    with torch.no_grad():
        feat = model.encode_image(image)
        image_features.append(feat)

image_features = torch.cat(image_features, dim=0)

# Normalizar vectores
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features  /= text_features.norm(dim=-1, keepdim=True)

# Convertir a NumPy
image_np = image_features.cpu().numpy()
text_np  = text_features.cpu().numpy()

# Calcular matriz de similitud entre imágenes y textos
similarity_matrix = cosine_similarity(image_np, text_np)

# Crear DataFrame para visualización
labels = ["Black seafood rice", "Dog", "Chips", "Coliseum", "Stadium"]
df_sim = pd.DataFrame(similarity_matrix, index=[f"{l} (img)" for l in labels], columns=labels)

# Imprimir tabla redondeada
print("\nMatriz de similitud coseno entre imágenes y textos:")
print(df_sim.round(4))


# División y recomposición de datos de Nueva York para procesamiento eficiente por lotes

Dado que el procesamiento de las reseñas con embeddings multimodales (especialmente la verificación de enlaces de imágenes y el cálculo de embeddings con CLIP) requería gran cantidad de memoria y tiempo, se observó que al trabajar con archivos grandes el sistema se volvía inestable o se colgaba. Para evitar estos problemas, se adoptó una estrategia de procesamiento por partes.

Primero, el dataset original se dividió en tres fragmentos de tamaño similar. Cada fragmento se procesó de forma independiente, lo que permitió verificar los enlaces de las imágenes y generar los embeddings sin sobrecargar la memoria del entorno de ejecución (por ejemplo, en Kaggle o Colab).

Una vez completado el procesamiento parcial, los archivos resultantes se volvieron a recombinar para reconstruir el conjunto completo. Este proceso se aplicó tanto a los embeddings multimodales (texto + imagen) como a los embeddings textuales obtenidos en distintas fases.

Este enfoque incremental resultó ser más robusto, escalable y menos propenso a errores, facilitando la generación y consolidación de los datos embebidos para su uso posterior en modelos de recomendación.

In [ ]:
import pandas as pd

# Cargar dataset original
df = pd.read_pickle("/kaggle/input/reviewspartny/reviews_part3.pkl")

# Dividir en 3 partes
n = len(df)
split1 = n // 3
split2 = 2 * n // 3

df_part1 = df.iloc[:split1].reset_index(drop=True)
df_part2 = df.iloc[split1:split2].reset_index(drop=True)
df_part3 = df.iloc[split2:].reset_index(drop=True)

# Guardar
df_part1.to_pickle("/kaggle/working/reviews_3_a.pkl")
df_part2.to_pickle("/kaggle/working/reviews_3_b.pkl")
df_part3.to_pickle("/kaggle/working/reviews_3_c.pkl")

print("Dataset dividido en 3 partes y guardado en /kaggle/working/")

In [ ]:
import pandas as pd

# Grupo 1
df1a = pd.read_pickle("/kaggle/working/reviews_out_ny1a.pkl")
df1b = pd.read_pickle("/kaggle/working/reviews_out_ny1b.pkl")
df1c = pd.read_pickle("/kaggle/working/reviews_out_ny1c.pkl")
df1 = pd.concat([df1a, df1b, df1c], ignore_index=True)
df1.to_pickle("/kaggle/working/reviews_outemb_ny1.pkl")

# Grupo 2
df2a = pd.read_pickle("/kaggle/working/reviews_out_ny2a.pkl")
df2b = pd.read_pickle("/kaggle/working/reviews_out_ny2b.pkl")
df2c = pd.read_pickle("/kaggle/working/reviews_out_ny2c.pkl")
df2 = pd.concat([df2a, df2b, df2c], ignore_index=True)
df2.to_pickle("/kaggle/working/reviews_outemb_ny2.pkl")

# Grupo 3
df3a = pd.read_pickle("/kaggle/working/reviews_out_ny3a.pkl")
df3b = pd.read_pickle("/kaggle/working/reviews_out_ny3b.pkl")
df3c = pd.read_pickle("/kaggle/working/reviews_out_ny3c.pkl")
df3 = pd.concat([df3a, df3b, df3c], ignore_index=True)
df3.to_pickle("/kaggle/working/reviews_outemb_ny3.pkl")

print("Conjuntos unidos y guardados como reviews_out_ny1/2/3.pkl")

In [ ]:
import pandas as pd

# Lista de archivos a unir
files = [
    "/kaggle/input/ny-emb-fin/reviews_out_ny1a_text.pkl",
    "/kaggle/input/ny-emb-fin/reviews_out_ny1b_text.pkl",
    "/kaggle/input/ny-emb-fin/reviews_out_ny1c_text.pkl",
    "/kaggle/input/ny-emb-fin/reviews_out_ny2a_text.pkl",
    "/kaggle/input/ny-emb-fin/reviews_out_ny2b_text.pkl",
    "/kaggle/input/ny-emb-fin/reviews_out_ny2c_text.pkl",
    "/kaggle/input/ny-emb-fin/reviews_out_ny3a_text.pkl",
    "/kaggle/input/ny-emb-fin/reviews_out_ny3b_text.pkl",
    "/kaggle/input/ny-emb-fin/reviews_out_ny3c_text.pkl"
]

# Leer y concatenar todos
dfs = [pd.read_pickle(f) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Guardar resultado combinado
df.to_pickle("/kaggle/working/reviews_ny_full_emb.pkl")
print("✅ Pickles unidos en reviews_ny_full.pkl")